<a href="https://colab.research.google.com/github/aidenrosebush/ESC472-2023/blob/main/viewshed_los_fast.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# -*- coding: utf-8 -*-
"""Viewshed_LOS_fast.ipynb

Automatically generated by Colaboratory.

Original file is located at
    https://colab.research.google.com/github/aidenrosebush/ESC472-2023/blob/main/Viewshed_LOS_fast.ipynb
"""

import tcod
import numpy as np
import matplotlib.pyplot as plt

def getTestPoints(map, point, direction, r):
  '''
  Inputs: 
  map (any 2D array of the same size we want to consider for building height or elevation maps)
  point (coordinates of the ad in terms of indices on the map array)
  direction (a vector pointing in the direction the ad faces, in terms of indices of the array. Need not be a unit vector)
  r (distance visibility criterion for the ad)

  Output:
  testPoints (1D list of index pairs corresponding to all the points which are within the distance visibility criterion of the ad, 
  and which are on the same side as the ad. We use a full 180 degree sweep of points which might provide visibility of the ad)

  '''
  testPoints = [] #points on the same side as the ad is facing
  for i in range(0, len(map), 1): 
    for j in range(0, len(map[0]), 1):
      relativeTestPoint = np.array([i-point[0],j-point[1]]) #encode the magnitude and direction of a testpoint relative to the ad location
      #if the point is on the right side of the billboard and is within visible range
      if np.dot(relativeTestPoint, direction) >= 0 and np.sqrt(np.dot(relativeTestPoint, relativeTestPoint)) <= r: #criteria for possible visibilty
        testPoints += [np.array([i,j])]
  return testPoints


def plotColoredMap(coloredMap):
  '''
  Inputs:
  coloredMap (2D array where points the ad is visible from are assigned to 1, all other assigned to 0)
  Outputs:
  coloredMap (unchanged)
  '''
  fig, ax = plt.subplots() #Plot the visible points using matplotlib
  ax.imshow(coloredMap, origin="lower")
  ax.set_title('Test Points')
  return coloredMap


def getColoredMap(map, testPoints):
  '''
  Inputs: 
  map (2D array of elevations in the size we want to consider)
  Outputs:
  coloredMap (2D array where all potentially visible points in the map array are set to 1. This array is modified
  to only include visible points based on elevation data and sightlines)
  '''
  coloredMap = np.zeros((len(map), len(map[0]))) #initialize as zeros
  for testPoint in testPoints:
    coloredMap[testPoint[0], testPoint[1]] = 1 #assign values in coloredMap to 1 as needed
  return coloredMap


def plotMap(map):
  '''
  Inputs:
  map (some 2D elevation array, which this function plots)
  Outputs:
  None
  '''
  fig, ax = plt.subplots() #create new plot
  im = ax.imshow(map, origin='lower')
  ax.set_title('Elevation Map')
  fig.colorbar(im, ax=ax, label='Elevation') #plot the elevation data


def fitSightLine(map, point, testPoint, elevation):
  '''
  Inputs:
  map (2D array; elevation map, buildings and ground heights considered)
  point (indices of the position of the ad)
  testPoint (current point which we want to compute a hypothetical sightline for, connecting it to the ad position)
  elevation (height of the ad)
  Outputs:
  m (The slope of the line connecting the ad to the test point (as calculated by the test point's elevation, stored in the map))
  '''
  # y-y0 = m*(r) (slope calculated by setting the ad location as the origin and computing the radius of the test point)
  m = (map[testPoint[0]][testPoint[1]]-elevation)/(np.sqrt(sum((point-testPoint)**2)))
  return m


def getViewshed(map, point, direction, r, elevation):
  '''
  Inputs:
  map (2D array; elevation map, buildings and ground heights considered)
  point (indices of the position of the ad)
  direction (a vector pointing in the direction the ad faces, in terms of indices of the array. Need not be a unit vector)
  r (distance visibility criterion)
  elevation (height of the ad)
  Outputs:
  coloredMap (2D array where points the ad is visible from are assigned to 1, all other assigned to 0)
  '''
  point = np.array(point)
  t = getTestPoints(map, point, direction, r)
  coloredMap = getColoredMap(map, t)

  line = []

  y = 0
  
  for testPoint in t:
    y += 1
    if testPoint[0] != point[0] and testPoint[1] != point[1] or coloredMap[testPoint[0]][testPoint[1]] == 1: #don't consider the ad location itself
      line = tcod.los.bresenham(point,testPoint) #get grid coordinates to search through
      max = -1 #elevation data must bottom out at 0
      m = 0 #store slope of top sightline
      m_test = 0 #store slope of sightline of test point when the test point is not above any point yet seen
      for i in range(1, len(line), 1):
        if coloredMap[line[i][0], line[i][1]] == 1: #only consider points which are still marked as potentially visible
          if i == 1:
            m = fitSightLine(map, point, line[1], elevation)
          else:
            m_test = fitSightLine(map, point, line[i], elevation)
            if m_test >= m: #if the slope is larger (more positive or less negative, we can see over everything in between)
              m = m_test #keep track of maximum slope
            else:
              coloredMap[line[i][0], line[i][1]] = 0 #if the slope is smaller, the point can't see the ad
      
  return coloredMap


def plotVisiblePoints(coloredMap, building_array = None):
  '''
  Inputs: 
  coloredMap (2D array where points the ad is visible from are assigned to 1, all other assigned to 0)
  building_array (2D array of only building heights)
  Outputs: 
  None

  '''
  fig, ax = plt.subplots()
  if building_array is not None:
    ax.imshow(building_array > 0, cmap='gray') #superimpose buildings of nonzero height over elevation array
  ax.imshow(coloredMap, alpha=coloredMap/np.amax(coloredMap), origin="lower") #plot visible points
  ax.set_title('Visible Points')
  return

def detectBuildings(buildingMap, coloredMap):
  '''
  Inputs: 
  buildingMap (2D elevation array of buildings)
  coloredMap (2D array denoting visible points)
  Outputs:
  coloredMap (2D array denoting visible points, with all indices corresponding to buildings removed)
  '''
  for i in range(len(coloredMap)):
    for j in range(len(coloredMap[i])):
      if buildingMap[i][j] != 0: #if the buildingMap suggests a building is at those indices (with nonzero height), mark those points as not visible
        coloredMap[i][j] = 0
  return coloredMap

#--- all code below this point is for testing purposes only -----
def addBuildingSquare(map, x_left,x_right,y_bottom,y_top, height):
  '''
  Inputs:
  map (2D elevation array, being constructed)
  x_left (left edge indices of building to be added )
  Outputs:
  map (modified to include new buildings)
  '''

  for x in range(x_left,x_right,1):
    for y in range(y_bottom, y_top, 1):
      map[x][y] = height #reset heights to represent a building
  return map

if __name__=="__main__":
  #test code to generate an elevation and building map, and test it
  scale = 1
  eMap = np.random.uniform(0, 10, (100*scale,100*scale))
  eMap = np.zeros((100*scale, 100*scale))
  buildingMap = np.zeros((100*scale, 100*scale))
  buildingMap = addBuildingSquare(buildingMap, 30*scale,40*scale,30*scale,40*scale,25)
  buildingMap = addBuildingSquare(buildingMap, 70*scale,75*scale,80*scale,85*scale,30)
  buildingMap = addBuildingSquare(buildingMap, 20*scale,30*scale,50*scale,60*scale,22)
  buildingMap = addBuildingSquare(buildingMap, 20*scale,35*scale,75*scale,80*scale,25)
  map = eMap + buildingMap
  point = [scale*50,scale*50]
  direction = [-1,1]
  r = 60*scale
  elevation = 60
  m = getViewshed(map, point, direction, r, elevation)
  m = detectBuildings(buildingMap, m)
  plotVisiblePoints(m)